# **Crop**

In [ ]:
import cv2
import sys
import os.path
import glob

output = glob.glob('/content/drive/Shareddrives/인수 플젝/webtoon_crop/*/*.png')

def make_file_name(filename):
    folder = filename.split('/')[-2]
    file = filename.split('/')[-1].split('.')[0]
    return folder + "_" + file

def detect(filename, cascade_file = "/content/drive/Shareddrives/인수 플젝/face-detected_2/lbpcascade_animeface.xml"):
    if not os.path.isfile(cascade_file):
        raise RuntimeError("%s: not found" % cascade_file)

    cascade = cv2.CascadeClassifier(cascade_file)
    image = cv2.imread(filename, cv2.IMREAD_COLOR)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)
    
    faces = cascade.detectMultiScale(gray,
                                     scaleFactor = 1.1,
                                     minNeighbors = 3,
                                     minSize = (3, 3))
    for index, (x, y, w, h) in enumerate(faces):
      if w != h:
        print(filename)
        continue
      cv2.imwrite(make_file_name(filename)+ "_" + str(index) + ".png", image[y: y+h, x: x+w, :])

for filename in output:
    detect(filename)
   



# **UpScaling**


In [ ]:
from zipfile import ZipFile
file_name = '/content/drive/Shareddrives/인수 플젝/Waifu2x-master.zip'

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

In [ ]:
import os
import glob
import cv2
import matplotlib.pyplot as plt
import numpy as np

from utils.prepare_images import *
from Models import *
from torchvision.utils import save_image

model_cran_v2 = CARN_V2(color_channels=3, mid_channels=64, conv=nn.Conv2d,
                        single_conv_size=3, single_conv_group=1,
                        scale=2, activation=nn.LeakyReLU(0.1),
                        SEBlock=True, repeat_blocks=3, atrous=(1, 1, 1))
                        
model_cran_v2 = network_to_half(model_cran_v2)
checkpoint = "/content/drive/Shareddrives/인수 플젝/model_check_points/CRAN_V2/CARN_model_checkpoint.pt"
model_cran_v2.load_state_dict(torch.load(checkpoint, 'cpu'))
# if use GPU, then comment out the next line so it can use fp16. 
model_cran_v2 = model_cran_v2.float() 

def upscaling(img_dir):
  
  img_name = img_dir.split('/')[-1]
  #demo_img = image
  img = Image.open(img_dir).convert("RGB")

  if img.size[0]<128:
    return
  elif img.size[0]>256:
    print(img_name)
    res = img.resize((256,256))
    res.save("/content/drive/Shareddrives/인수 플젝/img512/"+img_name)
  else:
    print(img_name)
    tmp = img.resize((128,128))
    #tmp = to_tensor(tmp).unsqueeze(0) 

    img_splitter = ImageSplitter(seg_size=64, scale_factor=2, boarder_pad_size=3)
    img_patches = img_splitter.split_img_tensor(tmp, scale_method=None, img_pad=0)
    with torch.no_grad():
        out = [model_cran_v2(i) for i in img_patches]
    res = img_splitter.merge_img_tensor(out)
    save_image(res,"/content/drive/Shareddrives/인수 플젝/img512/"+img_name)
  #final = torch.cat([img_t, img_upscale])
  #save_image(final, 'out.png', nrow=2)
  #print(img_upscale.shape,final.shape)
  #plt.imshow(img_upscale[0].permute(1, 2, 0))
  #plt.show()
  

for i in glob.glob("/content/drive/Shareddrives/인수 플젝/face-detected_2/*.png"):
  upscaling(i)
# image = "input_image.png"

# upscaling(image)